In [1]:
import os

In [2]:
pwd

'c:\\Prajesh\\personal\\ML_learning\\my_work\\kubeflow_app\\kubeflow_ML_app\\ML_flow_app\\research'

In [3]:
cd ..

c:\Prajesh\personal\ML_learning\my_work\kubeflow_app\kubeflow_ML_app\ML_flow_app


In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class ModelTrainerConfig:
    root_dir: Path
    train_data_path: Path
    test_data_path: Path
    model_name: str
    n_estimators: int
    max_depth: int
    target_col: str

In [5]:
from mlFlowProject.constants import *
from mlFlowProject.utils.common import read_yaml, create_directories

In [6]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH
    ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_model_trainer_config(self) -> ModelTrainerConfig:
        config = self.config.model_trainer
        params = self.params.RandomForestRegressor
        schema = self.schema.TARGET_COLUMN
        
        create_directories([config.root_dir])
        
        model_trainer_config = ModelTrainerConfig(
            root_dir = config.root_dir,
            train_data_path = config.train_data_path,
            test_data_path = config.test_data_path,
            model_name = config.model_name,
            n_estimators = params.n_estimators,
            max_depth = params.max_depth,
            target_col = schema.name
        )
        
        return model_trainer_config

In [19]:
import pandas as pd
import os
from mlFlowProject import logger
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
import optuna
import joblib
import numpy as np
import yaml

In [23]:
class ModelTrainer:
    def __init__(self, config: ModelTrainerConfig):
        self.config = config
    
    def hp_tune(self, trial, xtrain, ytrain):
        # crit = trial.suggest_categorical("criterion", ["entropy"])
        n_est = trial.suggest_int("n_estimators", 2, 200, log = True)
        m_depth = trial.suggest_int("max_depth", 1, 100, log = True)
        rfr = RandomForestRegressor(n_estimators = n_est, max_depth = m_depth)
        
        score = cross_val_score(rfr, xtrain, ytrain, cv = 3)
        accuracy = score.mean()
        return accuracy

    def train(self):
        train_data = pd.read_csv(self.config.train_data_path)
        test_data = pd.read_csv(self.config.test_data_path)
        
        x_train = np.array(train_data['Open']).reshape(-1, 1)
        x_test = np.array(test_data['Open']).reshape(-1, 1)
        y_train = np.array(train_data[[self.config.target_col]]).reshape(-1)
        y_test = np.array(test_data[[self.config.target_col]]).reshape(-1)
        
        print(x_train.shape, y_train.shape)
        study = optuna.create_study(direction="maximize")
        # study.optimize(self.hp_tune, (n_trials=25, x_train, y_train))
        study.optimize(lambda trial: self.hp_tune(trial, x_train, y_train), n_trials=25)
        n_estimators = study.best_params['n_estimators']
        max_depth = study.best_params['max_depth']
        rfr = RandomForestRegressor(
            n_estimators= n_estimators,
            max_depth= max_depth
        )
        
        rfr.fit(x_train, y_train)
        
        joblib.dump(rfr, os.path.join(self.config.root_dir, self.config.model_name))
        
        if self.config.n_estimators != n_estimators or self.config.max_depth != max_depth:
            with open(PARAMS_FILE_PATH, 'r') as f:
                tuned_params = yaml.safe_load(f)
            tuned_params["RandomForestRegressor"]["n_estimators"] = n_estimators
            tuned_params["RandomForestRegressor"]["max_depth"] = max_depth
            with open(PARAMS_FILE_PATH, 'w') as f:
                yaml.dump(tuned_params, f, default_flow_style=False)

In [24]:
try:
    config_manager = ConfigurationManager()
    model_trainer_config = config_manager.get_model_trainer_config()
    model_trainer = ModelTrainer(config = model_trainer_config)
    model_trainer.train()
except Exception as e:
    raise e

[2023-08-07 15:42:13,449: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-07 15:42:13,450: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-07 15:42:13,452: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-08-07 15:42:13,453: INFO: common: created directory at: artifacts]
[2023-08-07 15:42:13,454: INFO: common: created directory at: artifacts/model_trainer]


[I 2023-08-07 15:42:13,467] A new study created in memory with name: no-name-cd332aac-1c87-4041-adba-b163708ea778


(5205, 1) (5205,)


[I 2023-08-07 15:42:13,997] Trial 0 finished with value: 0.9973430920474698 and parameters: {'n_estimators': 56, 'max_depth': 87}. Best is trial 0 with value: 0.9973430920474698.
[I 2023-08-07 15:42:14,623] Trial 1 finished with value: 0.9238731216407077 and parameters: {'n_estimators': 153, 'max_depth': 2}. Best is trial 0 with value: 0.9973430920474698.


56 87
